<a href="https://colab.research.google.com/github/crismunoz/Tutorial_Detectron/blob/main/Detectron2_infernece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime
!pip install -U kora

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import sys
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
import matplotlib.pyplot as plt
from kora.drive import upload_public
from IPython.display import HTML
def show_image(img):
  plt.figure(figsize=(10,10))
  plt.imshow(img)
  plt.axis('off')
  plt.show()

Inferencia em uma imagem
1. Baixar uma imagem de exemplo
2. abrir imagem usando cv2
3. mostrar imagem

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -O input.jpg

im = cv2.imread("./input.jpg")
show_image(im[:,:,::-1])

Configuração e definição do modelo:

In [ ]:
cfg = get_cfg()
config_file="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(config_file))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_file)  # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
#classes = outputs["instances"].pred_classes.cpu().numpy()
#masks = outputs['instances'].pred_masks.cpu().numpy()
#boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy()
#[metadata.thing_classes[c] for c in classes]

Visualização de resultado:

In [ ]:
v = Visualizer(im[:, :, ::-1], metadata, scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
show_image(v.get_image())

# Camera Test

In [ ]:
#TODO:  USAR CODE SNIPPETS (CAMERA CAPTURE)
#TODO: REALIZAR INFERENCIA

# Video Test

Se apresenta um código que abre um vídeo e itera sobre todos os frames. Utiliza-se este código base para implemtar a inferência sobre um vídeo.

https://www.pexels.com/video/cars-on-highway-854671/

In [ ]:
url = upload_public('Cars On Highway.mp4')
HTML(f"""<video src={url} width=1000 controls/>""")

In [ ]:
video_input='video.mp4'
video_output='inference.mp4'

# Configuration input video
cap = cv2.VideoCapture(video_input)
frame_width, frame_height = (640,480)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Configuration output video
fourcc =cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(video_output,fourcc, 20.0, (frame_width, frame_height))

# TODO: DEFINIR MODELO

for id_frame in tqdm(range(0,num_frames,10)):
  cap.set(cv2.CAP_PROP_POS_FRAMES,id_frame)
  flag, frame = cap.read()

  if flag:
    frame = cv2.resize(frame, (frame_width, frame_height))

    #TODO: INFERENCIA 

    out_frame = # TODO: VISUALIZACAO
    
    out.write(out_frame[:,:,::-1].astype('uint8'))

cap.release()
out.release()